1x1 Comparison Test:

In [4]:
from ccdc.crystal import PackingSimilarity
from ccdc.io import CrystalReader
crystal_reader = CrystalReader(r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_5679_predicted_crystals\T2_5679_predicted_crystals\job_00001.cif', format='cif')
cryst0 = crystal_reader[0]
crystal_reader = CrystalReader(r'C:\Users\Clamfighter\Machine_Learning_Project\my_env\Masters\T2_5679_predicted_crystals\T2_5679_predicted_crystals\job_00011.cif', format='cif')
cryst1 = crystal_reader[0]
PS = PackingSimilarity()
comp = PS.compare(cryst0,cryst1)
print(comp.rmsd)
print(comp.nmatched_molecules)

0.21640852422677578
2


Experimentral Crystal Comparison

In [5]:
import os
import numpy as np
import pandas as pd
from ccdc.io import CrystalReader
from ccdc.crystal import PackingSimilarity
import time
'''
Initial run 9x9 with duplicate comparisons: 516.1s
Run without duplicate comparison 270.4s
'''
directory = r'T2_9_experimental_crystals'
PS = PackingSimilarity()#Packing shell object with default parameters
i=0
cifs = [] #get name of each CIF file
for filename in os.listdir(directory):
    cifs.append(filename)
#Prepare Output Matrices dynamically by the lenght of the file
rmsd_matrix = np.empty((len(cifs),len(cifs)))
rmsd_matrix[:] = np.nan
matchedMolecules_matrix = np.empty((len(cifs),len(cifs)))
matchedMolecules_matrix[:] = np.nan
cols = cifs
starttime = time.time()
print(starttime)
for i in range(len(cifs)):
    '''
    compare each CIF file to all other CIF files
    do not compare the same files
    once a file is compared, take it out the list so it isn't compared to other files
    '''
    outter_cif = cifs[i]
    crystal_reader = CrystalReader(directory+'\\'+outter_cif)
    outter_crystal = crystal_reader[0]
    for j in range(len(cifs)):
        inner_cif = cifs[j]
        if inner_cif != outter_cif and inner_cif != "": #don't compare id not in list or same crystal
            crystal_reader = CrystalReader(directory+'\\'+inner_cif)
            inner_crystal = crystal_reader[0]
            comp = PS.compare(outter_crystal,inner_crystal)
            rmsd = comp.rmsd
            matched_molecules = comp.nmatched_molecules
            rmsd_matrix[i][j] = rmsd
            matchedMolecules_matrix[i][j] = matched_molecules
        elif inner_cif == outter_cif:#default values if same crystal
            rmsd_matrix[i][j] = 0
            matchedMolecules_matrix[i][j] = 15
    cifs[i] = ""
    i+=1
rmsd_df = pd.DataFrame(rmsd_matrix, columns=cols, index=cols)#store values in DFs
matchedMolecule_df = pd.DataFrame(matchedMolecules_matrix, columns=cols, index=cols)
endtime = time.time()
print(rmsd_df)
print(matchedMolecule_df)
print(endtime)
print(endtime - starttime)
cols = []#bug where cols is empty after loop so need to refill to add columns and rows names
for filename in os.listdir(directory):
    cols.append(filename)
rmsd_df.index = cols
rmsd_df.columns = cols
matchedMolecule_df.index = cols
matchedMolecule_df.columns = cols
rmsd_df.to_csv(r"fullComparisonOutputRMSD.csv")
matchedMolecule_df.to_csv(r"fullComparisonOutputMatchedMolecules.csv")

0.2
16.0
1631831864.9768095
                                                                             \
  0.0  0.008312  0.003178  0.034960  0.451601  0.439513  0.186498  0.426636   
  NaN  0.000000  0.007424  0.034131  0.452272  0.440206  0.186618  0.426416   
  NaN       NaN  0.000000  0.033081  0.453086  0.440990  0.186980  0.427092   
  NaN       NaN       NaN  0.000000  0.462409  0.450179  0.186841  0.427149   
  NaN       NaN       NaN       NaN  0.000000  0.054768  0.246310  0.664421   
  NaN       NaN       NaN       NaN       NaN  0.000000  0.239013  0.654570   
  NaN       NaN       NaN       NaN       NaN       NaN  0.000000  0.273618   
  NaN       NaN       NaN       NaN       NaN       NaN       NaN  0.000000   
  NaN       NaN       NaN       NaN       NaN       NaN       NaN       NaN   

            
  0.183327  
  0.183868  
  0.183369  
  0.183587  
  0.434904  
  0.426874  
  0.231050  
  0.575751  
  0.000000  
                                                   

Comparing Experimental and Predicted Crystals

In [17]:
import os
import numpy as np
import pandas as pd
from ccdc.io import CrystalReader
from ccdc.crystal import PackingSimilarity

outterdirectory = r'T2_5679_predicted_crystals'
innerdirectory = r'T2_9_experimental_crystals'
PS = PackingSimilarity()
i=0
outtercifs = []
innercifs = []
#define two lists of CIFS
for filename in os.listdir(outterdirectory):
    outtercifs.append(filename)
for filename in os.listdir(innerdirectory):
    innercifs.append(filename)
#Define matrices dynamically by number of files in directories
rmsd_matrix = np.empty((len(outtercifs),len(innercifs)))
rmsd_matrix[:] = np.nan
matchedMolecules_matrix = np.empty((len(outtercifs),len(innercifs)))
matchedMolecules_matrix[:] = np.nan
rows = outtercifs
cols = innercifs
for i in range(len(outtercifs)):#for each outter CIF
    outter_cif = outtercifs[i]
    crystal_reader = CrystalReader(outterdirectory+'\\'+outter_cif)#create crystal object
    outter_crystal = crystal_reader[0]
    for j in range(len(innercifs)):#for each outter CIF and Inner CIF pair
        inner_cif = innercifs[j]
        try:
            if inner_cif != outter_cif and inner_cif != "":
                crystal_reader = CrystalReader(innerdirectory+'\\'+inner_cif)#create crystal object
                inner_crystal = crystal_reader[0]
                comp = PS.compare(outter_crystal,inner_crystal)#compare the two crystals
                rmsd = comp.rmsd#RMSD
                matched_molecules = comp.nmatched_molecules#MM
                rmsd_matrix[i][j] = rmsd#Save to matrix
                matchedMolecules_matrix[i][j] = matched_molecules#Save to matrix
            elif inner_cif == outter_cif:#If they're the same CIF, use default values, don't compare
                rmsd_matrix[i][j] = 0
                matchedMolecules_matrix[i][j] = 15
        except RuntimeError:#some CIF files have data errors
            rmsd_matrix[i][j] = None
            matchedMolecules_matrix[i][j] = None
    outtercifs[i] = ""
    i+=1
#create dataframes from matrices that will be saved to file
rows=[]#bug where rows does not persist past loop
for filename in os.listdir(outterdirectory):
    rows.append(filename)
rmsd_df = pd.DataFrame(rmsd_matrix, columns=cols, index=rows)
matchedMolecule_df = pd.DataFrame(matchedMolecules_matrix, columns=cols, index=rows)
print(rmsd_df)
print(matchedMolecule_df)
rmsd_df.to_csv(r"OutputRMSD.csv")
matchedMolecule_df.to_csv(r"OutputMatchedMolecules.csv")

                                  job_00001.cif  job_00011.cif
DEBXIT04_1478364_T2gamma.cif           0.176380       0.125317
DEBXIT05_1520228_T2beta_240K.cif       0.432749       0.408948
DEBXIT06_1520229_T2beta_293K.cif       0.424928       0.401201
job_00001.cif                          0.000000       0.216409
NAVXUG_1478356_T2alpha.cif             0.227971       1.105594
SEMDIA_1520227_T2delta_240K.cif        0.597740       0.478748
T2epsilon.cif                          0.127940       0.266690
                                  job_00001.cif  job_00011.cif
DEBXIT04_1478364_T2gamma.cif                3.0            2.0
DEBXIT05_1520228_T2beta_240K.cif            3.0            2.0
DEBXIT06_1520229_T2beta_293K.cif            3.0            2.0
job_00001.cif                              15.0            2.0
NAVXUG_1478356_T2alpha.cif                  3.0            2.0
SEMDIA_1520227_T2delta_240K.cif             5.0            2.0
T2epsilon.cif                              15.0        